### Hugging Face orientado a LLM

1 . Instalación y configuración

Para comenzar a trabajar con la biblioteca Hugging Face Transformers, primero debes instalarla junto con sus dependencias. Utiliza pip para instalarlo

In [ ]:
!pip install transformers datasets torch optuna


2 . Carga y uso de modelos preentrenados

Hugging Face ofrece una amplia variedad de modelos preentrenados que puedes cargar y usar fácilmente. 


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# Cargar el tokenizador y el modelo GPT-2
tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")
modelo_gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")

# Entrada de texto y generación
input_text = "Once upon a time"
inputs = tokenizer_gpt2(input_text, return_tensors="pt")
outputs = modelo_gpt2.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=50,
    pad_token_id=tokenizer_gpt2.eos_token_id
)
print(tokenizer_gpt2.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# Ejemplo de clasificación con BERT
# In[5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "bert-base-uncased"
tokenizer_cls = AutoTokenizer.from_pretrained(model_name)
# Especificar num_labels=5 para el dataset yelp_review_full (5 clases)
modelo_cls = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

3 . Fine-Tuning de modelos

Fine-tuning es el proceso de ajustar un modelo preentrenado en un conjunto de datos específico para mejorar su desempeño en una tarea concreta. A continuación se muestra un ejemplo básico de cómo realizar fine-tuning en un conjunto de datos personalizado.

**Prepara el conjunto de datos**

Para este ejemplo, usaremos un conjunto de datos de Hugging Face Datasets.

In [ ]:
# Cargar el conjunto de datos y preparar el fine-tuning
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")

**Tokenización del conjunto de datos**
Es necesario tokenizar los datos para que el modelo los entienda.

In [ ]:
# Tokenización del conjunto de datos
def tokenize_function(examples):
    return tokenizer_cls(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


**Dividir el conjunto de datos**

Dividimos el conjunto de datos en conjuntos de entrenamiento y validación.

In [ ]:
# Dividir en entrenamiento y evaluación
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))


**Preparar el modelo para fine-tuning**

Cargamos el modelo preentrenado y lo preparamos para la tarea de clasificación de texto.

In [ ]:
from transformers import TrainingArguments, Trainer

# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    optim="adamw_torch"  # Usar la implementación de AdamW de PyTorch
)



**Configurar el entrenamiento**

Configuramos los parámetros de entrenamiento y usamos Trainer para entrenar el modelo.

In [ ]:
# Configurar el entrenamiento
trainer = Trainer(
    model=modelo_cls,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
#Entrenar el modelo de clasificación
trainer.train()

4 . Evaluación del modelo

Después del entrenamiento, evaluamos el modelo para ver cómo se desempeña en el conjunto de datos de prueba.



In [ ]:
# Evaluación del modelo
eval_results = trainer.evaluate()
print("Resultados de evaluación:", eval_results)


5 . Uso del modelo fine-tuned

Finalmente, podemos usar el modelo entrenado para hacer predicciones en nuevos datos.

In [ ]:
# Uso del modelo fine-tuned
inputs_example = tokenizer_cls("Este es un ejemplo.", return_tensors="pt")
outputs_example = modelo_cls(**inputs_example)
print("Predicción del modelo fine-tuned:", outputs_example)

6 . Guardado y carga del modelo

Es importante guardar el modelo entrenado para su uso futuro. Aquí te mostramos cómo guardar y cargar el modelo.

In [ ]:
# Guardar el modelo y el tokenizer
modelo_cls.save_pretrained("./fine_tuned_model")
tokenizer_cls.save_pretrained("./fine_tuned_model")

**Cargar el modelo**

In [ ]:
# Cargar el modelo y el tokenizer guardados
modelo_cls = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model", num_labels=5)
tokenizer_cls = AutoTokenizer.from_pretrained("./fine_tuned_model")

7 . **Ajuste de hiperparámetros**

El ajuste de hiperparámetros puede mejorar significativamente el rendimiento del modelo. Esto implica experimentar con diferentes valores de hiperparámetros como la tasa de aprendizaje, el tamaño del batch, etc.


[Optuna](https://optuna.org/) es una biblioteca para la optimización automática de hiperparámetros.



In [ ]:
# 7. Ajuste de hiperparámetros con Optuna
import optuna

def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    num_train_epochs = trial.suggest_int('num_train_epochs', 1, 5)
    
    training_args_trial = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
    )
    
    trainer_trial = Trainer(
        model=modelo_cls,
        args=training_args_trial,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )
    
    trainer_trial.train()
    eval_results = trainer_trial.evaluate()
    return eval_results['eval_loss']

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)
print("Mejores hiperparámetros:", study.best_params)

### Ejercicios


#### Ejercicio 1: Cambiar el modelo de generación de texto

- **Objetivo:** Modificar el ejemplo de generación de texto para utilizar un modelo distinto, por ejemplo, `"EleutherAI/gpt-neo-125M"`.
- **Pasos sugeridos:**
  - Reemplaza la carga de `gpt2` por el modelo `gpt-neo-125M`.
  - Ajusta el parámetro `max_length` y experimenta con diferentes longitudes de salida.
  - Prueba distintas frases de entrada y analiza la coherencia y creatividad de las respuestas.
- **Puntos a considerar:**  
  - ¿Cómo afecta el cambio de modelo a la calidad de la generación?
  - ¿Qué diferencias encuentras en cuanto a tiempos de respuesta y uso de memoria?

#### Ejercicio 2: Implementar un pipeline de análisis de sentimientos

- **Objetivo:** Usar la API de `pipeline` de Hugging Face para crear un sistema de análisis de sentimientos.
- **Pasos sugeridos:**
  - Utiliza `from transformers import pipeline` para cargar el pipeline de `"sentiment-analysis"`.
  - Crea una lista de textos en español (o en el idioma que prefieras) y pásalos al pipeline.
  - Muestra y analiza los resultados obtenidos, comparándolos con casos esperados.
- **Extensión:**  
  - Integra una interfaz simple en consola o web para que el usuario pueda ingresar su propio texto y recibir el análisis de sentimiento.

#### Ejercicio 3: Crear un sistema RAG para preguntas y respuestas

- **Objetivo:** Implementar un sistema de generación de respuestas usando Retrieval Augmented Generation (RAG).
- **Pasos sugeridos:**
  - Investiga y utiliza un modelo RAG preentrenado, por ejemplo, `"facebook/rag-token-base"` o `"facebook/rag-sequence-base"`.
  - Prepara un pequeño corpus de documentos (pueden ser textos almacenados localmente o extraídos de alguna fuente) que sirvan como base de conocimiento.
  - Diseña un flujo en el que, dado un prompt o pregunta, se recupere información relevante del corpus y se combine con la generación del modelo.
- **Puntos a considerar:**  
  - ¿Cómo afecta la calidad del corpus a la respuesta final?
  - ¿Qué mejoras se podrían hacer en la fase de recuperación (retrieval) para optimizar el sistema?

#### Ejercicio 4: Fine-tuning de un modelo para clasificación de texto en otro dominio

- **Objetivo:** Realizar fine-tuning de un modelo preentrenado (por ejemplo, BERT) para una tarea de clasificación en un dominio distinto al de Yelp.
- **Pasos sugeridos:**
  - Escoge otro dataset (por ejemplo, de reseñas de productos, noticias, etc.) y carga los datos usando `load_dataset` de Hugging Face.
  - Adapta la función de tokenización según las características del nuevo dataset.
  - Divide el dataset en conjuntos de entrenamiento y validación.
  - Configura y entrena el modelo utilizando `Trainer`.
  - Evalúa el rendimiento del modelo y discute posibles mejoras.
- **Extensión:**  
  - Compara el desempeño de distintos modelos preentrenados (por ejemplo, BERT vs. RoBERTa) en la misma tarea.

#### Ejercicio 5: Optimización de hiperparámetros con Optuna

- **Objetivo:** Aplicar técnicas de ajuste de hiperparámetros para mejorar el rendimiento del modelo de clasificación.
- **Pasos sugeridos:**
  - Define un espacio de búsqueda para parámetros como la tasa de aprendizaje (`learning_rate`), el número de épocas (`num_train_epochs`), tamaño del batch, etc.
  - Modifica la función `objective` para que entrene el modelo con distintos conjuntos de hiperparámetros y devuelva la métrica de evaluación (por ejemplo, `eval_loss`).
  - Ejecuta la optimización con un número determinado de pruebas (`n_trials`).
  - Analiza los mejores hiperparámetros encontrados y discute por qué podrían ser óptimos para el problema.
- **Puntos a considerar:**  
  - ¿Cómo influye el número de trials en el resultado final?
  - ¿Qué estrategias podrías implementar para acelerar el proceso de optimización?

#### Ejercicio 6: Crear una aplicación interactiva con un pipeline LLM

- **Objetivo:** Desarrollar una aplicación sencilla (por ejemplo, utilizando Streamlit o Flask) que utilice un pipeline de generación de texto para simular un chatbot o asistente interactivo.
- **Pasos sugeridos:**
  - Crea una interfaz web donde el usuario pueda escribir un prompt.
  - Utiliza el pipeline de `"text-generation"` para generar una respuesta en tiempo real.
  - Añade funcionalidades como el historial de conversación o ajustes dinámicos (por ejemplo, ajustar `max_length` o la temperatura del muestreo).
- **Extensión:**  
  - Integra opciones para cambiar el modelo de generación y compara las respuestas entre distintos modelos.


In [ ]:
## Tus respuestas